In [2]:
from code_tokenizer import *

# code = """
# posicao.Trigger == "DEACTIVATION" || posicao.Trigger == "POWER_LOSS"
# """

code = """
const my_list = [1, 2, 3, 4, 5]
let min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)
"""

print(color_by_char_type(code))

parsed = math_layer(code)

for i in parsed:
    if(i[0] == 'word' and i[1] == 'for'):
        parameters = i[2][1]
        block = i[3][1]
        if(not block is list):
            block = [block]
        
        for j in parameters:
            print(j)
        for j in block:
            print(j)


const my_list = [1, 2, 3, 4, 5]
let min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)

[['assignment', [['word', 'i'], [['number', '1']]]]]
[['word', 'i'], ['acess_group', [['word', 'my_list'], ['word', 'length']]], ['symbol', '<']]
[['increment', ['word', 'i']]]
['word', 'if', ['attached_group', [['word', 'my_list', ['array_call', ['word', 'i']]], ['word', 'min'], ['symbol', '<']]], ['attached_block', [['assignment', [['word', 'min'], [['word', 'my_list', ['array_call', ['word', 'i']]]]]]]]]
